In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
import numpy.linalg as npl
from scipy.linalg import toeplitz
np.set_printoptions(precision=2)

In [2]:
# generate data - M features, N samples
N = 10
M = 2

# x1 = np.arange(N)
# x2 = x1**3
# X = np.array([x1, x2]).T # NxM

np.random.seed(1)
X = np.random.randn(N,M) # NxM
# plt.plot(X[:,0], X[:,1], 'C0.')
D = np.array([[npl.norm(X[i]-X[j], ord=2) for i in range(N)] for j in range(N)])
# print(D)

imap = {} # map array index to matrix index
i=0
d = np.array([])
for r in range(N):
    for c in range(r+1, N):
        imap[i] = (r, c)
        d = np.append(d, D[r,c])
        i += 1
        
idx = np.argsort(d)
d = d[idx]
# print(d)

S = [imap[idx[i]] + imap[idx[i+1]] for i in range(len(idx)-1)]
# print(S)

In [4]:
k = 1 # force difference between comparisons
l = 1 # reg param for rank

# inequality constraints
A = []
for (i,j,k,l) in S:
    a = np.zeros((N,N))
    a[k,k] += 1
    a[k,l] += -1
    a[l,l] += -1
    a[l,l] += 1
    a[i,i] += -1
    a[i,j] += 1
    a[j,i] += 1
    a[j,j] += -1
    A.append(a)
    
K_ = cp.Variable((N,N), symmetric=True)
E = cp.Variable(len(S))

constraints = []
constraints = [K_ >> 0] # PSD
constraints += [cp.sum(K_) == 0] # center
# constraints += [E[i] >= 0 for i in range(len(S))] # slack
# constraints += [E >= 0]
# constraints += [cp.trace(A[i]@K_) >= (k - E[i]) for i in range(len(S))]

# prob = cp.Problem(cp.Minimize(cp.sum(E) + l*cp.trace(K_)), constraints)
cost = 0
for i in range(len(S)):
    cost += E[i] + l*cp.trace(K_)
    constraints += [E[i] >= 0]
    constraints += [cp.trace(A[i]@K_) >= (k - E[i])]
prob = cp.Problem(cp.Minimize(cost), constraints)
prob.solve()

219.99921738824722

In [9]:
lams, V = npl.eigh(K_.value)
print(npl.eigh(X.T@X)[0])
print(lams)
X_ = np.array([lams[i]*V[i] for i in range(2)]).T

# plt.figure(figsize=(12,4))
# plt.subplot(121)
# plt.plot(X[:,0], X[:,1], 'C0.')
# plt.subplot(122)
# plt.plot(X_[:,0], X_[:,1], 'C1.')

[ 4.26 20.29]
[-2.56e-06 -2.00e-06 -1.87e-06 -8.33e-07 -6.54e-07  2.03e-07  8.23e-07
  9.74e-07  1.45e-06  2.50e-06]


In [6]:
D_ = np.array([[npl.norm(X_[i]-X_[j], ord=2) for i in range(N)] for j in range(N)])
imap_ = {} # map array index to matrix index
i=0
d_ = np.array([])
for r in range(N):
    for c in range(r+1, N):
        imap_[i] = (r, c)
        d_ = np.append(d_, D_[r,c])
        i += 1
        
# S_ = [imap_[idx_[i]] + imap_[idx_[i+1]] for i in range(len(idx_)-1)]
succ = np.zeros(len(S))
success = np.zeros(len(S))
for s in range(len(S)):
    i,j,k,l = S[s]
    succ[s] = D[i,j] <= D[k,l]
    success[s] = D_[i,j] <= D_[k,l]

# print(S)
preserved = (len(S) - np.count_nonzero(succ-success))/len(S)
print(preserved)

0.6136363636363636


In [7]:
E.value

array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])